In [ ]:
import pandas as pd

def load_excel(file):
    if file is None:
        raise ValueError("No file provided")
    return pd.read_excel(file)

: 

In [ ]:
file_path = "/home/tatas/ChromaQuantExtension/src/ChromaQuantExtension/MBPR053_Auto_Breakdown_20241230.xlsx"
df = load_excel(file_path)

In [9]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,NaN,Sample Name,Reactor Name,Catalyst Type,Catalyst Amount (mg),Plastic Type,Plastic Amount (mg),Reaction Temperature (C),Quench Temperature (C),Reaction Pressure (psi),...,Reactor Volume (mL),Remaining solids (mg),Injected CO2 (mL),Reaction Time,NaN,Total mass source,Mass (mg),NaN,NaN,NaN
1,NaN,MBPR053_01,Tall,Ru/C+BEA,195.1,PE4k Sigma-Aldrich,1015.6,280,29.6,250,...,35,917.6,2.5,72,NaN,Overall breakdown,83.249157,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Compound Type Breakdown,83.249157,NaN,NaN,NaN
3,NaN,FID RT,FID Area,MS RT,Compound Name,Formula,Match Factor,Compound Source,Compound Type Abbreviation,Compound Type,...,NaN,NaN,NaN,NaN,NaN,Carbon Number Breakdown,83.249157,NaN,NaN,NaN
4,NaN,2.424,80.6288,2.3951,"1,2-Ethanediol",C2H6O2,60.713327,Automatically assigned by comparing FID and MS...,O,Other,...,NaN,NaN,NaN,NaN,NaN,Compound Type + Carbon Number Breakdown,83.249157,NaN,NaN,NaN


In [ ]:
import pandas as pd

# Define the exact locations of the tables in each sheet
table_locations = {
    "Sheet1": {
        "Table1": {"start_row": 2, "start_col": 1, "end_row": 5, "end_col": 4},  # A2:D5
        "Table2": {"start_row": 8, "start_col": 1, "end_row": 10, "end_col": 3},  # A8:C10
    },
    "Sheet2": {
        "Table1": {"start_row": 3, "start_col": 2, "end_row": 6, "end_col": 5},  # B3:E6
    },
    # Add more sheets and tables as needed
}

def extract_table_from_sheet(sheet_df, start_row, start_col, end_row, end_col):
    """
    Extract a table from a DataFrame based on the specified row and column range.
    """
    # Adjust for 0-based indexing in pandas
    start_row -= 1
    start_col -= 1
    end_row -= 1
    end_col -= 1

    # Extract the table
    table = sheet_df.iloc[start_row:end_row + 1, start_col:end_col + 1]
    return table

def extract_tables_from_excel(file, table_locations):
    """
    Extract specific tables from an Excel file based on predefined locations.
    """
    extracted_tables = {}

    # Load the Excel file
    xls = pd.ExcelFile(file)

    for sheet_name, tables in table_locations.items():
        # Load the sheet into a DataFrame
        sheet_df = pd.read_excel(xls, sheet_name=sheet_name, header=None)

        for table_name, location in tables.items():
            # Extract the table
            table = extract_table_from_sheet(
                sheet_df,
                location["start_row"],
                location["start_col"],
                location["end_row"],
                location["end_col"],
            )
            # Store the table in the dictionary
            extracted_tables[f"{sheet_name}_{table_name}"] = table

    return extracted_tables

# Example usage
file_path = r"C:\Users\tatas\Downloads\MBPR053_Auto_Breakdown_20241230.xlsx"
extracted_tables = extract_tables_from_excel(file_path, table_locations)

# Print the extracted DataFrames
for table_name, df in extracted_tables.items():
    print(f"Table: {table_name}")
    print(df)
    print("\n")

### MH - March 6

In [ ]:
import pandas as pd

def print_sheet_names(file_path):
    """Loads an Excel file and prints the names of all sheets."""
    xls = pd.ExcelFile(file_path)
    print("Sheet names:", xls.sheet_names)

# Call the function with your file
file_path = "MBPR053_Auto_Breakdown_20241230.xlsx"  # Replace with your actual file path
print_sheet_names(file_path)

In [ ]:
import pandas as pd

def extract_tables_from_sheet(file_path, sheet_name):
    """Extracts multiple tables from a given Excel sheet and returns them as separate DataFrames."""
    
    # Load the entire sheet into a DataFrame
    xls = pd.ExcelFile(file_path)
    sheet_df = pd.read_excel(xls, sheet_name, header=None)  # Load without headers

    # Define table ranges (adjust as needed)
    tables = {
        "main": sheet_df.iloc[4:295, 1:15],  # B5:O295
        "summary": sheet_df.iloc[7:14, 16:19],  # Q8:S14
        "rxn_info": sheet_df.iloc[1:3, 1:21],  # B2:U3
        "breakdown": sheet_df.iloc[1:6, 22:24],  # W2:X6
        "postprocessed": sheet_df.iloc[7:42, 20:27],  # U8:AA42
        "CnMass": sheet_df.iloc[15:50, 16:18],  # Q16:R50
    }

    # Assign column names from the first row of each table
    for key in tables:
        tables[key].columns = tables[key].iloc[0]  # Set first row as header
        tables[key] = tables[key][1:].reset_index(drop=True)  # Drop original header row

    return tables

# Example usage
file_path = "MBPR053_Auto_Breakdown_20241230.xlsx"
sheet_name = "Liquid FID"  # Replace with the actual sheet name

tables = extract_tables_from_sheet(file_path, sheet_name)

# Display the first few rows of each extracted table
for name, df in tables.items():
    print(f"\n{name} table:")
    print(df.head())